In [1]:
import math

class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op    # For graphviz, debugging, etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Only support int/float powers!"
        out = Value(self.data**other, (self,), f'**{other}')
        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward
        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')
        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
        
    def __neg__(self):    # -self
        return self * -1

    def __radd__(self, other):    # other + self
        return self + other

    def __sub__(self, other):    # self - other
        return self + (-other)

    def __rsub__(self, other):    # other - self
        return other + (-self)

    def __rmul__(self, other):    # other * self
        return self * other

    def __truediv__(self, other):    # self / other
        return self * other**-1

    def __rtruediv__(self, other):    # other / self
        return other * self**-1

    def __repr__(self):
#       return f"Value(data={self.data}, grad={self.grad})"
        return f"Value(data={self.data})"

    def backward(self):
        # Topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

In [2]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges



def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [3]:
import random

class Neuron:
    
    def __init__(self, in_num):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(in_num)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
#       out = act.relu()
        return out

    def parameters(self):
        return self.w + [self.b]



class Layer:
    
    def __init__(self, in_num, out_num):
        self.neurons = [Neuron(in_num) for _ in range (out_num)]

    def __call__(self, x):
        outs = [neuron(x) for neuron in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [param for neuron in self.neurons for param in neuron.parameters()]



class MLP:        # Multi-layer perceptron

    def __init__(self, in_num, outs_num):
        sz = [in_num] + outs_num
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(outs_num))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [param for layer in self.layers for param in layer.parameters()]        

In [4]:
n = MLP(3, [4, 4, 1])    # 2 hidden layers with 4 perceptrons each
epoch_count = 20
learning_rate = 0.09

In [5]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0] ]
ys = [1.0, -1.0, -1.0, 1.0]

In [6]:
for k in range(epoch_count):
    
    # Forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    
    # Backward pass
    for p in n.parameters(): p.grad = 0.0
    loss.backward()
    
    # Update
    for p in n.parameters():
        p.data += -learning_rate * p.grad    

    print("Epoch:", k, " Loss:", loss.data)

Epoch: 0  Loss: 4.992039307083247
Epoch: 1  Loss: 3.8985905201369704
Epoch: 2  Loss: 4.904865082854977
Epoch: 3  Loss: 0.9662912278936067
Epoch: 4  Loss: 0.2883617587706834
Epoch: 5  Loss: 0.1418099309695227
Epoch: 6  Loss: 0.11038791028768881
Epoch: 7  Loss: 0.08916797117743513
Epoch: 8  Loss: 0.07410302525739257
Epoch: 9  Loss: 0.06297732618921895
Epoch: 10  Loss: 0.05449250166449458
Epoch: 11  Loss: 0.04784774784550323
Epoch: 12  Loss: 0.04252736187590711
Epoch: 13  Loss: 0.038186786725249326
Epoch: 14  Loss: 0.034588414178453
Epoch: 15  Loss: 0.031563838970816
Epoch: 16  Loss: 0.02899081868272735
Epoch: 17  Loss: 0.026778739106660558
Epoch: 18  Loss: 0.024859174240496815
Epoch: 19  Loss: 0.02317959436393785


In [7]:
ypred

[Value(data=0.9249471792182691),
 Value(data=-0.9933631660905072),
 Value(data=-0.9079923667168066),
 Value(data=0.9049357253752681)]

In [8]:
# for layer in self.layers 
# for param in layer.parameters()

for layer in n.layers:
    print("Layer params: ", end= '')
    for neur in layer.neurons:
        print(len(neur.w + [neur.b]), end= ' ')
    print('')
print("Total:", len(n.parameters()))

Layer params: 4 4 4 4 
Layer params: 5 5 5 5 
Layer params: 5 
Total: 41


In [9]:
n.parameters()    # Model parameters learned

[Value(data=-0.16508745829690386),
 Value(data=0.8482939045351171),
 Value(data=-1.182098270337857),
 Value(data=0.5516189867956568),
 Value(data=0.2897606696803398),
 Value(data=-1.0676863861714878),
 Value(data=0.09372921621165549),
 Value(data=-0.2720152780363272),
 Value(data=-0.7375340619098627),
 Value(data=-0.7355574945743973),
 Value(data=0.5086558343922897),
 Value(data=-0.7870595619593468),
 Value(data=-0.14242096087993286),
 Value(data=-0.5140102156045772),
 Value(data=1.0861760449858693),
 Value(data=0.6001539789992669),
 Value(data=0.6178416525561902),
 Value(data=-0.6410105782699504),
 Value(data=-0.42020402679752944),
 Value(data=0.43957753733182237),
 Value(data=0.1434194156909264),
 Value(data=-0.4145147094322809),
 Value(data=0.821822422630469),
 Value(data=-0.34498808400184106),
 Value(data=0.5611095537174459),
 Value(data=-0.13088952614402388),
 Value(data=0.422059208216207),
 Value(data=-0.750021219314138),
 Value(data=0.6936554910155232),
 Value(data=-1.3854268348